In [ ]:
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import  model as md
import  dataprocess as DP
from torch.autograd import *
import torch.optim as optim
import torch.nn.functional as F
import argparse
import json
from json import encoder
import sys
if sys.getdefaultencoding() != 'utf-8':
    reload(sys)
    sys.setdefaultencoding('utf8')
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

def model_training(paramters):
    
    '''取数据'''
    train_seq, test_seq = DP.DataGenerate(paramters)  # 取出数据len(train_seq[1][1]) 
    
    t0=time.time()

    '''选择模型'''
    if paramters.model_mode  == 'LSTMpred':
        if paramters.pre_train==True:
            model = md.LSTMpred(paramters).cuda()
        elif paramters.pre_train==False:
            model=torch.load(paramters.model_mode+'_model.pkl')
        
    elif  paramters.model_mode  == 'TrajPreSimple':
        model = md.TrajPreSimple(paramters).cuda()
    print('model:',model)

    '''损失函数'''
    loss_function=nn.MSELoss()
    # loss_function =nn.NLLLoss()
    
    '''优化器'''
    if paramters.optimizer=='Adam':
        optimizer = optim.Adam(model.parameters(), lr=paramters.learning_rate)
    else:
        optimizer = optim.SGD(model.parameters(), lr=paramters.learning_rate)
        
    
    metrics = {'time': [], 'epoch': [], 'Loss': [],'test_avg_loss': []}
    
    '''开始训练'''
    print('Start training...')
    count=0
    for epoch in range(paramters.epoch):
        for seq, outs in train_seq:
            count=count+1
            seq = torch.FloatTensor(seq).cuda()
            outs = torch.FloatTensor(outs).cuda()
            outs=torch.unsqueeze(outs,1)
            
            modout = model(seq)
            loss = loss_function(modout, outs)
            
            optimizer.zero_grad()   
            loss.backward()
            optimizer.step()
            
            #每一百步保存一次模型
            if count%500==0:
                 torch.save(model,paramters.model_mode+'_model.pkl')
                 print( 'Epoch:', epoch, 'Loss:', loss.cpu().data.numpy())


        test_avg_loss=model_test(test_seq)
    
        tim1 = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
        metrics['time'].append(tim1)
        metrics['epoch'].append(int(epoch))
        metrics['Loss'].append(float(loss.cpu().data.numpy()))
        metrics['test_avg_loss'].append(float(test_avg_loss))

        print(tim1,'  Epoch:',epoch,'  Loss:',loss.cpu().data.numpy(),'  Test_avg_loss:',test_avg_loss)


    t1 = time.time() - t0
    print('Total training time: ', t1)
    torch.save(model,paramters.model_mode+'_model.pkl')
    json.dump({'metrics': metrics}, fp=open('Training_process'+tim1+'.rs', 'w'), indent=4)
    

    

def model_test(test_seq):
    
    model=torch.load(paramters.model_mode+'_model.pkl')
    
    loss_function=nn.MSELoss()
    
    predDat = []
    total=0
    loss_total = []
    
    
    
    for seq, trueVal in test_seq:
        seq = torch.FloatTensor(seq).cuda()
        trueVal = torch.FloatTensor(trueVal).cuda()
        
        modout = model(seq)
        loss = loss_function(modout, trueVal)
        
        loss_total.append(loss.cpu().data.numpy())
        
#     print('Test_avg_loss',sum(loss_total) / len(loss_total))
    
    return sum(loss_total) / len(loss_total)
        

#         predDat.append(model(seq)[-1].data.cpu().numpy()[0])        
#     fig = plt.figure()
#     plt.title('Adam')
#     plt.plot(dataset)
    
#     plt.plot(predDat)
#     plt.show()

   ####简单sin(x)回归问题
#     train_seq, test_seq, dataset= DP.Generate_data_demo(paramters)  # 取出数据len(train_seq[1][1])


if __name__ == '__main__':
    '''参数设计'''
    parser = argparse.ArgumentParser()
    parser.add_argument('--hidden_size', type=int, default=512, help="hidden size")
    parser.add_argument('--time_step', type=int, default=96, help="time step")
    parser.add_argument('--optimizer', type=str, default='Adam', choices=['Adam', 'SGD'])
    parser.add_argument('--traning_data_rate', type=int, default=0.8, help="traning_data_rate")
    parser.add_argument('--learning_rate', type=int, default=2 * 1e-3, help="leanning_rate")
    parser.add_argument('--batch_size', type=int, default=1, help="batch size")
    parser.add_argument('--dropout_p', type=int, default=0.4, help="dropout_rate")
    parser.add_argument('--rnn_type', type=str, default='LSTM', choices=['LSTM', 'GRU', 'RNN'])
    parser.add_argument('--use_cuda', type=bool, default=True, choices=[True, False])
    parser.add_argument('--pre_train', type=bool, default=False, choices=[True, False])
    parser.add_argument('--nomalization', type=bool, default=True, choices=[True, False])
    parser.add_argument('--model_mode', type=str, default='LSTMpred',
                        choices=['LSTMpred', 'TrajPreSimple', 'attn_avg_long_user', 'attn_local_long'])    
    
    parser.add_argument('--input_size', type=int, default=80, help="input size")
    parser.add_argument('--output_size', type=int, default=80, help="output size")
    
    '''参数'''
    parser.add_argument('--data_path', type=str,default=u'dataset100000.mat', help="data path")
    parser.add_argument('--epoch', type=int, default=200, help="epoch size")
    
    paramters = parser.parse_args(['--input_size', '80'])

    np.random.seed(3)
    torch.manual_seed(3)
    
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    
    '''训练'''
    model_training(paramters)
    

#     '''测试'''
#     train_seq, test_seq = DP.DataGenerate(paramters)  # 取出数据len(train_seq[1][1]) 
#     model_test(test_seq)


model: LSTMpred(
  (lstm): LSTM(80, 512)
  (fc): Linear(in_features=512, out_features=80, bias=True)
  (dropout): Dropout(p=0.4)
)
Start training...
Epoch: 0 Loss: 0.0034718954
Epoch: 0 Loss: 0.005872478
Epoch: 0 Loss: 0.0095985625
Epoch: 0 Loss: 0.0054289224
Epoch: 0 Loss: 0.0058747316
Epoch: 0 Loss: 0.010563695
Epoch: 0 Loss: 0.0042005144
Epoch: 0 Loss: 0.004465743
Epoch: 0 Loss: 0.0076700947
Epoch: 0 Loss: 0.0033572037
Epoch: 0 Loss: 0.0049337153
Epoch: 0 Loss: 0.0076271133
Epoch: 0 Loss: 0.0036772552
Epoch: 0 Loss: 0.0044117905
Epoch: 0 Loss: 0.011339738
Epoch: 0 Loss: 0.0047556646
Epoch: 0 Loss: 0.0039553377
Epoch: 0 Loss: 0.0062582134
Epoch: 0 Loss: 0.0039519845
Epoch: 0 Loss: 0.006607207
Epoch: 0 Loss: 0.010257976
Epoch: 0 Loss: 0.004888856
Epoch: 0 Loss: 0.0052190707
Epoch: 0 Loss: 0.0070184185
Epoch: 0 Loss: 0.0026560077
Epoch: 0 Loss: 0.0055887797
Epoch: 0 Loss: 0.00841349
Epoch: 0 Loss: 0.0026438206
Epoch: 0 Loss: 0.0039407015
Epoch: 0 Loss: 0.01144816
Epoch: 0 Loss: 0.00488

Epoch: 1 Loss: 0.0060894834
Epoch: 1 Loss: 0.0035273142
Epoch: 1 Loss: 0.0027044467
Epoch: 1 Loss: 0.0048672855
Epoch: 1 Loss: 0.004765631
Epoch: 1 Loss: 0.004178195
Epoch: 1 Loss: 0.006493912
Epoch: 1 Loss: 0.005064803
Epoch: 1 Loss: 0.004031698
Epoch: 1 Loss: 0.0044143563
Epoch: 1 Loss: 0.0038917423
Epoch: 1 Loss: 0.0033522707
Epoch: 1 Loss: 0.0057412786
Epoch: 1 Loss: 0.004893019
Epoch: 1 Loss: 0.002849466
Epoch: 1 Loss: 0.0053557395
Epoch: 1 Loss: 0.005218382
Epoch: 1 Loss: 0.0034337144
Epoch: 1 Loss: 0.0078164665
Epoch: 1 Loss: 0.00333786
Epoch: 1 Loss: 0.005043617
Epoch: 1 Loss: 0.0045672795
Epoch: 1 Loss: 0.0040343567
Epoch: 1 Loss: 0.0030512996
Epoch: 1 Loss: 0.007577474
Epoch: 1 Loss: 0.0042950334
Epoch: 1 Loss: 0.0033164315
Epoch: 1 Loss: 0.005129566
Epoch: 1 Loss: 0.0032827142
Epoch: 1 Loss: 0.004007906
2019-08-02 10:53:24   Epoch: 1   Loss: 0.0076513877   Test_avg_loss: 0.005090170272896184
Epoch: 2 Loss: 0.009191886
Epoch: 2 Loss: 0.003966743
Epoch: 2 Loss: 0.005384086
Epo

Epoch: 3 Loss: 0.0045944364
Epoch: 3 Loss: 0.0061829137
Epoch: 3 Loss: 0.0048684306
Epoch: 3 Loss: 0.004061801
Epoch: 3 Loss: 0.0048349705
Epoch: 3 Loss: 0.0039170957
Epoch: 3 Loss: 0.003845964
Epoch: 3 Loss: 0.004983916
Epoch: 3 Loss: 0.0049675866
Epoch: 3 Loss: 0.0034677354
Epoch: 3 Loss: 0.0067493753
Epoch: 3 Loss: 0.0054049143
Epoch: 3 Loss: 0.003854984
Epoch: 3 Loss: 0.006302023
Epoch: 3 Loss: 0.004673118
Epoch: 3 Loss: 0.0042540715
Epoch: 3 Loss: 0.0049467892
Epoch: 3 Loss: 0.004494126
Epoch: 3 Loss: 0.0028962817
Epoch: 3 Loss: 0.0048192916
Epoch: 3 Loss: 0.004329841
Epoch: 3 Loss: 0.0027571628
Epoch: 3 Loss: 0.008065991
Epoch: 3 Loss: 0.004943843
Epoch: 3 Loss: 0.0042314935
Epoch: 3 Loss: 0.0049093235
Epoch: 3 Loss: 0.0034009614
Epoch: 3 Loss: 0.0036741316
Epoch: 3 Loss: 0.004971554
Epoch: 3 Loss: 0.0043068393
Epoch: 3 Loss: 0.0027613775
Epoch: 3 Loss: 0.0060415445
Epoch: 3 Loss: 0.0040476667
Epoch: 3 Loss: 0.0031226627
Epoch: 3 Loss: 0.0044283536
Epoch: 3 Loss: 0.0032814809
Epo

Epoch: 5 Loss: 0.005181426
Epoch: 5 Loss: 0.0036648926
Epoch: 5 Loss: 0.0058431393
Epoch: 5 Loss: 0.0038472866
Epoch: 5 Loss: 0.0026991544
Epoch: 5 Loss: 0.0073416396
Epoch: 5 Loss: 0.0058040908
Epoch: 5 Loss: 0.0038870363
Epoch: 5 Loss: 0.0071687386
Epoch: 5 Loss: 0.004305782
Epoch: 5 Loss: 0.0029981625
Epoch: 5 Loss: 0.0064944336
Epoch: 5 Loss: 0.00615053
Epoch: 5 Loss: 0.004410968
Epoch: 5 Loss: 0.0063948995
Epoch: 5 Loss: 0.0041986085
Epoch: 5 Loss: 0.0027887989
Epoch: 5 Loss: 0.006003246
Epoch: 5 Loss: 0.0037227322
Epoch: 5 Loss: 0.0033942335
Epoch: 5 Loss: 0.004752073
Epoch: 5 Loss: 0.004130054
Epoch: 5 Loss: 0.0030606762
Epoch: 5 Loss: 0.0044693933
Epoch: 5 Loss: 0.004492733
Epoch: 5 Loss: 0.00329717
Epoch: 5 Loss: 0.0044754227
Epoch: 5 Loss: 0.0041969665
Epoch: 5 Loss: 0.0022692694
Epoch: 5 Loss: 0.0041058436
Epoch: 5 Loss: 0.003573976
Epoch: 5 Loss: 0.0031336602
Epoch: 5 Loss: 0.005649379
Epoch: 5 Loss: 0.0030180218
Epoch: 5 Loss: 0.002772043
Epoch: 5 Loss: 0.0040345015
Epoch:

Epoch: 7 Loss: 0.0054674232
Epoch: 7 Loss: 0.0030429359
Epoch: 7 Loss: 0.0032724645
Epoch: 7 Loss: 0.0039234124
Epoch: 7 Loss: 0.0050522424
Epoch: 7 Loss: 0.00362178
Epoch: 7 Loss: 0.0062085907
Epoch: 7 Loss: 0.004404211
Epoch: 7 Loss: 0.0027503413
Epoch: 7 Loss: 0.00873238
Epoch: 7 Loss: 0.0030104988
Epoch: 7 Loss: 0.004306409
Epoch: 7 Loss: 0.004945568
Epoch: 7 Loss: 0.003817179
Epoch: 7 Loss: 0.0033813969
Epoch: 7 Loss: 0.0057666553
Epoch: 7 Loss: 0.004407379
Epoch: 7 Loss: 0.0030009677
Epoch: 7 Loss: 0.005055046
Epoch: 7 Loss: 0.0042312676
Epoch: 7 Loss: 0.0040157796
Epoch: 7 Loss: 0.006219325
Epoch: 7 Loss: 0.0047109574
Epoch: 7 Loss: 0.0031975
Epoch: 7 Loss: 0.0054492573
Epoch: 7 Loss: 0.0035134063
Epoch: 7 Loss: 0.002595156
Epoch: 7 Loss: 0.0060826624
Epoch: 7 Loss: 0.003531096
Epoch: 7 Loss: 0.0042805015
Epoch: 7 Loss: 0.006774018
Epoch: 7 Loss: 0.0038891435
Epoch: 7 Loss: 0.0031639982
Epoch: 7 Loss: 0.0050159665
Epoch: 7 Loss: 0.0037450367
Epoch: 7 Loss: 0.0037808367
Epoch: 7 

...

Epoch: 63 Loss: 0.42556635
Epoch: 63 Loss: 0.09048482
Epoch: 63 Loss: 0.07257418
Epoch: 63 Loss: 0.0046658413
Epoch: 63 Loss: 0.021006372
Epoch: 63 Loss: 0.007051508
Epoch: 63 Loss: 0.016549934
Epoch: 63 Loss: 0.009363863
Epoch: 63 Loss: 0.20114264
Epoch: 63 Loss: 0.022034157
Epoch: 63 Loss: 0.16723609
Epoch: 63 Loss: 0.02398832
Epoch: 63 Loss: 0.0040408624
Epoch: 63 Loss: 0.10186925
Epoch: 63 Loss: 0.10046581
Epoch: 63 Loss: 0.044030614
Epoch: 63 Loss: 0.047361802
Epoch: 63 Loss: 0.050629474
Epoch: 63 Loss: 0.020260451
Epoch: 63 Loss: 0.029975215
Epoch: 63 Loss: 0.0052266954
Epoch: 63 Loss: 0.6588597
2019-08-03 04:15:19   Epoch: 63   Loss: 0.17494681   Test_avg_loss: 0.27654938196231243
Epoch: 64 Loss: 0.09610065
Epoch: 64 Loss: 0.07023002
Epoch: 64 Loss: 0.19730148
Epoch: 64 Loss: 0.09685359
Epoch: 64 Loss: 0.019895006
Epoch: 64 Loss: 0.07299738
Epoch: 64 Loss: 1.2064252
Epoch: 64 Loss: 0.055423997
Epoch: 64 Loss: 0.18023983
Epoch: 64 Loss: 0.42123133
Epoch: 64 Loss: 0.87818927
Epoch

Epoch: 65 Loss: 0.0078882985
Epoch: 65 Loss: 0.07954357
Epoch: 65 Loss: 0.006459745
Epoch: 65 Loss: 0.38983172
Epoch: 65 Loss: 0.4873951
Epoch: 65 Loss: 0.05055837
Epoch: 65 Loss: 0.05452639
Epoch: 65 Loss: 0.028172174
Epoch: 65 Loss: 0.04955605
Epoch: 65 Loss: 0.051843382
Epoch: 65 Loss: 0.08233063
Epoch: 65 Loss: 0.005340987
Epoch: 65 Loss: 0.12985367
Epoch: 65 Loss: 0.051897895
Epoch: 65 Loss: 0.009157525
Epoch: 65 Loss: 0.038639225
Epoch: 65 Loss: 0.006365189
Epoch: 65 Loss: 0.006683906
Epoch: 65 Loss: 0.021551367
Epoch: 65 Loss: 0.89021397
Epoch: 65 Loss: 0.08949938
Epoch: 65 Loss: 0.07053032
Epoch: 65 Loss: 0.075033225
Epoch: 65 Loss: 0.008359296
Epoch: 65 Loss: 0.059119664
Epoch: 65 Loss: 0.119820334
Epoch: 65 Loss: 0.03707484
Epoch: 65 Loss: 0.6876758
Epoch: 65 Loss: 0.07880459
Epoch: 65 Loss: 0.076590165
Epoch: 65 Loss: 0.03535607
Epoch: 65 Loss: 0.22995827
Epoch: 65 Loss: 0.11791559
Epoch: 65 Loss: 0.056507308
Epoch: 65 Loss: 0.015655968
Epoch: 65 Loss: 0.017107673
Epoch: 65 

Epoch: 67 Loss: 0.022573719
Epoch: 67 Loss: 0.00755301
Epoch: 67 Loss: 0.90992564
Epoch: 67 Loss: 0.032072265
Epoch: 67 Loss: 0.03353522
Epoch: 67 Loss: 0.009720888
Epoch: 67 Loss: 0.4909907
Epoch: 67 Loss: 0.034958772
Epoch: 67 Loss: 0.22710033
Epoch: 67 Loss: 0.020672115
Epoch: 67 Loss: 0.18355341
Epoch: 67 Loss: 0.009250434
Epoch: 67 Loss: 0.027051369
Epoch: 67 Loss: 0.102278784
Epoch: 67 Loss: 0.0054837996
Epoch: 67 Loss: 0.06589754
Epoch: 67 Loss: 0.013151911
Epoch: 67 Loss: 0.025755852
Epoch: 67 Loss: 0.06715538
Epoch: 67 Loss: 0.06708355
Epoch: 67 Loss: 0.029492825
Epoch: 67 Loss: 0.032882143
Epoch: 67 Loss: 0.04262525
Epoch: 67 Loss: 0.021951508
Epoch: 67 Loss: 0.07800019
Epoch: 67 Loss: 0.009226106
Epoch: 67 Loss: 0.10319277
Epoch: 67 Loss: 0.4991784
Epoch: 67 Loss: 0.004877403
Epoch: 67 Loss: 0.08593821
Epoch: 67 Loss: 0.049804118
Epoch: 67 Loss: 0.016506976
Epoch: 67 Loss: 0.46645027
Epoch: 67 Loss: 0.48832774
Epoch: 67 Loss: 0.3973233
Epoch: 67 Loss: 0.051474594
Epoch: 67 L

Epoch: 69 Loss: 0.0840787
Epoch: 69 Loss: 0.0066985204
Epoch: 69 Loss: 0.041941233
Epoch: 69 Loss: 0.29270825
Epoch: 69 Loss: 0.042154282
Epoch: 69 Loss: 0.093065046
Epoch: 69 Loss: 0.02844286
Epoch: 69 Loss: 0.050754417
Epoch: 69 Loss: 0.024568863
Epoch: 69 Loss: 0.064692855
Epoch: 69 Loss: 0.030943796
Epoch: 69 Loss: 0.06581582
Epoch: 69 Loss: 0.57256466
Epoch: 69 Loss: 0.12467933
Epoch: 69 Loss: 0.012674367
Epoch: 69 Loss: 0.050046496
Epoch: 69 Loss: 0.025638977
Epoch: 69 Loss: 0.12175705
Epoch: 69 Loss: 0.14169803
Epoch: 69 Loss: 0.20569435
Epoch: 69 Loss: 0.01325689
Epoch: 69 Loss: 0.51594746
Epoch: 69 Loss: 0.009941396
Epoch: 69 Loss: 0.01116299
Epoch: 69 Loss: 0.005145802
Epoch: 69 Loss: 0.006679274
Epoch: 69 Loss: 0.0434326
Epoch: 69 Loss: 0.022110203
Epoch: 69 Loss: 0.0071061673
Epoch: 69 Loss: 0.2175875
Epoch: 69 Loss: 1.5272545
Epoch: 69 Loss: 0.047148377
Epoch: 69 Loss: 0.14912823
Epoch: 69 Loss: 0.009587112
Epoch: 69 Loss: 0.18092605
Epoch: 69 Loss: 0.06101172
Epoch: 69 Lo

Epoch: 71 Loss: 0.009222018
Epoch: 71 Loss: 0.17284659
Epoch: 71 Loss: 0.03741162
Epoch: 71 Loss: 0.018268695
Epoch: 71 Loss: 0.065037966
Epoch: 71 Loss: 0.18543324
Epoch: 71 Loss: 0.0478066
Epoch: 71 Loss: 0.6427363
Epoch: 71 Loss: 0.18119793
Epoch: 71 Loss: 0.0058619524
Epoch: 71 Loss: 0.37398824
Epoch: 71 Loss: 0.060200464
Epoch: 71 Loss: 0.05447827
Epoch: 71 Loss: 0.009195041
Epoch: 71 Loss: 0.043088358
Epoch: 71 Loss: 0.08160669
Epoch: 71 Loss: 0.04188686
Epoch: 71 Loss: 0.010556754
Epoch: 71 Loss: 0.022297908
Epoch: 71 Loss: 0.046975795
Epoch: 71 Loss: 0.015750501
Epoch: 71 Loss: 0.023659563
Epoch: 71 Loss: 0.5004622
Epoch: 71 Loss: 0.018696401
Epoch: 71 Loss: 0.045062214
Epoch: 71 Loss: 0.058458447
Epoch: 71 Loss: 0.00654856
Epoch: 71 Loss: 0.068452105
Epoch: 71 Loss: 0.0073118904
Epoch: 71 Loss: 0.3523438
Epoch: 71 Loss: 0.00577917
Epoch: 71 Loss: 0.0054896018
Epoch: 71 Loss: 0.24822941
Epoch: 71 Loss: 0.011791274
Epoch: 71 Loss: 0.03436982
Epoch: 71 Loss: 0.1397663
Epoch: 71 L